# 02_model_training_and_evaluation.ipynb

**Project**: Lumbar Spine Degenerative Classification  
**Description**: This notebook orchestrates the training and evaluation of the deep learning models described in the project.  

---

## Table of Contents
1. [Environment and Imports](#section1)  
2. [Configuration Loading](#section2)  
3. [Model Training](#section3)  
4. [Evaluation](#section4)

---

<a id="section1"></a>
## 1. Environment and Imports

Ensures the correct Python environment, updates paths if necessary, and imports relevant modules.

In [ ]:
import os
import sys
import torch
import glob

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

module_path = os.path.abspath(os.path.join("src"))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.model.train_model import train_model
from src.evaluate.evaluate_model import evaluate_model
from src.data.ingest_data import load_config

print("Environment diagnostics:")
print(f"  Working directory : {os.getcwd()}")
print(f"  Python executable : {sys.executable}")

<a id="section2"></a>
## 2. Configuration Loading

Retrieves training-related settings (e.g. classification mode, model architecture, etc.) from `config.yml`.

In [ ]:
CONFIG_PATH = "config.yml"
config = load_config(CONFIG_PATH)

print("Configuration loaded successfully.")
print("Project:", config["project"]["name"])
print("Description:", config["project"]["description"])
print("Classification Mode:", config["training"]["classification_mode"])
print("Selected Folder:", config["training"]["selected_tensor_folder"])

<a id="section3"></a>
## 3. Model Training

Calls the `train_model` function from `train_model.py`.  
This will:
- Load data from CSV
- Split into train/val/test sets
- Create datasets and dataloaders
- Build the selected neural network architecture
- Train with early stopping
- Save logs and best model in `./models/...`

In [ ]:
train_model(CONFIG_PATH)

<a id="section4"></a>
## 4. Evaluation

The `evaluate_model` function from `evaluate_model.py` allows evaluating on a specified split.
By default, it re-splits or tries to load the test set. Then it prints metrics (accuracy, confusion matrix, etc.)

In [ ]:
import glob

# Automatically find the last created folder.
model_folders = glob.glob("models/*_*_*")
model_folders = sorted(model_folders, key=os.path.getmtime)
latest_folder = model_folders[-1] if model_folders else None

print("Latest model folder:", latest_folder)

# The best model .pth is saved as "best_model.pth" inside that folder.
model_path = os.path.join(latest_folder, "best_model.pth") if latest_folder else None
print("Model path:", model_path if model_path and os.path.exists(model_path) else "No model found.")

if model_path and os.path.exists(model_path):
    evaluate_model(model_path=model_path, config_path=CONFIG_PATH, split="test")
else:
    print("[WARNING] No valid model .pth file found. Skipping evaluation.")